# Extrahieren von Text zum Social Media Archiv 
- https://democrats-intelligence.house.gov/social-media-content/
- ```pip install progressbar```

In [5]:
import textract
import re
import os
import pandas as pd
import progressbar
import time

## Mit einer Datei ausprobieren

- Einleseversuch

In [36]:
#Testing with one file
text = textract.process('files/P(1)0000054.pdf', method='pdfminer')
text = str(text).replace("\n\n", " ")
text = str(text).replace("\n", " ")
text

'Ad ID 374 Ad Text Join us because we care. Black matters. Ad Landing Page https://www.facebook.com/Black-Matters-1579673598947501 / Ad Targeting Location: United States: Baltimore (+20 km) Maryland; St. Louis (+20 km) Missouri Excluded Connections: Exclude people who like Black Matters Age: 18 - 65+ Language: English (UK) or English (US) Placements: News Feed on desktop computers or News Feed on mobile devices Ad Impressions 137 Ad Clicks 0 Ad Spend 44.87 RUB Ad Creation Date 06/10/15 02:59:53 AM PDT P(1)0000054 Redactions Completed at the Direction of Ranking Member of the US House Permanent Select Committee on Intelligence\x0cSuggested Page Black Matters Sponsored Join us because we care. Black matters. Black Matters Commun,ty 224,537 people like  nis. ,` Like Page P(1)0000055 Redactions Completed at the Direction of Ranking Member of the US House Permanent Select Committee on Intelligence\x0c'

- Suchen nach dem Adtext:

In [32]:
adtext = re.search("Ad Text.*Ad Landing Page", text)
adtext = re.search("Ad Text.*Ad Landing Page", text).group().replace("Ad Landing Page", "")
adtext = adtext.replace("Ad Text", "").replace("\\n", " ").strip()
adtext

'Join us because we care. Black matters.'

In [34]:
def get_location(text):
    if re.search("Location.*(Age)", text) != None:
        pattern = re.search("Location.*(Age)", text)
        loc = pattern.group().replace('Age','').replace('Location - Living In: ','')
    else:
        loc = 'N/A'
    return loc
get_location(text)

'Location: United States: Baltimore (+20 km) Maryland; St. Louis (+20 km) Missouri Excluded Connections: Exclude people who like Black Matters '

In [44]:
def get_date(text):
    if re.search("Creation Date.*(Redactions)", str(text)) != None:
        pattern = re.search("Creation Date.*(Redactions)", str(text))
        pattern = pattern.group().replace('Creation Date\\n\\n','')
        date = re.search("\d\d/\d\d/\d\d \d\d:\d\d:\d\d [A-Z][A-Z] [A-Z]{3}", pattern)
        if date != None:
            date = date.group()
    else: 
        date = 'N/A'
    return date
get_date(text)

'06/10/15 02:59:53 AM PDT'

In [46]:
def get_ad_klicks(text):
    if re.search("Ad Clicks.*", text) != None:
        klicks = re.search("Ad Clicks.*", text)
        klicks = klicks.group()[:25].replace(",", "")
        klicks = re.search("[0-9]+", klicks)
        if klicks != None:
            klicks = klicks.group()
    else:
        klicks = 'N/A'
    return klicks
get_ad_klicks(text)

'0'

In [47]:
def get_ad_impressions(text):
    if re.search("Ad Impressions.*", text) != None:
        imps = re.search("Ad Impressions.*", text)
        imps = imps.group()[:25].replace(",", "")
        imps = re.search("[0-9]+", imps)
        if imps != None:
            imps = imps.group()
    else:
        imps = 'N/A'
    return imps
get_ad_impressions(text)

'137'

In [49]:
def get_ad_spend(text):
    if re.search("Ad Spend.*", text) != None:
        adspend = re.search("Ad Spend.*", text)
        adspend = adspend.group()[:25].replace(",", "")
        adspend = re.search("[0-9]+.[0-9]+", adspend)
        if adspend != None:
            adspend = adspend.group()
    else:
        adspend = 'N/A'
    return adspend
get_ad_spend(text)

'44.87'

## Aufbau der Dateiliste

In [72]:
!ls

2.1 Importing Text from Everywhere.ipynb
2.2 Real world example with textract.ipynb
2.3 Named Entities Recognition.ipynb
2.4 Classifying Text.ipynb
2015-q2.zip
bild.jpg
example.docx
example.pdf
example.png
files
guardian.png


In [73]:
#Creating file list
lt = sorted(os.listdir("files"))

In [74]:
lst = []
for elem in lt:
    if ".pdf" in elem:
        lst.append(elem)

In [75]:
#lst = lst[0:4]

In [76]:
#Creating dictionary for:

#'Location', 'Number', 'Date', 'Interests',
#'Age', 'Language', 'Placements', 'Ad Clicks',
#'Ad Impressions', 'Ad Spend'

bar = progressbar.ProgressBar()
results = []

for elem,i in zip(lst, bar(range(len(lst)-1))):
    #print(elem)
    if 'ultimate_names.csv' in elem:
        continue
    
    else:
        
        text = textract.process('files/'+elem, method='pdfminer')
        text = str(text).replace("\n\n", " ")
        text = str(text).replace("\n", " ")

    
        location = get_location(text)
        date = get_date(text)
        klicks = get_ad_klicks(text)
        imps = get_ad_impressions(text)
        adspend = get_ad_spend(text)
        
        
        if re.search("Interests:.*", text) != None:
            interests = re.search("Interests:.*", text).group().split('\\n')[0]
        else:
            interests = 'N/A'
    
        if re.search("Age:.*", text) != None:
            age = re.search("Age:.*", text).group().split('\\n')[0]
        else:
            age = 'N/A'
        
        if re.search("Language:.*", text) != None:
            lang = re.search("Language:.*", text).group().split('\\n')[0]
        else:
            lang = 'N/A'
    
        if re.search("Placements:.*", text) != None:
            place = re.search("Placements:.*", text).group().split('\\n')[0]
        else:
            place = 'N/A'
    
        if re.search("Ad Text.*Ad Landing Page", text) != None:
            adtext = re.search("Ad Text.*Ad Landing Page", text).group().replace("Ad Landing Page", "")
            adtext = adtext.replace("Ad Text", "").replace("\\n", " ").strip()
        else:
            place = 'N/A'
        
        mini_dict = {'Location': loc,
                 'Number': elem,
                 'Date': date,
                 'Interests': interests,
                 'Age': age,
                 'Language': lang,
                 'Placements': place,
                 'Ad Text': adtext,
                 'Ad Clicks': klicks,
                 'Ad Impressions': imps,
                 'Ad Spend': adspend}
        
        results.append(mini_dict)


100% |########################################################################|


### Abspeichern im Dataframe

In [77]:
df = pd.DataFrame(results)

In [78]:
df

,Ad Clicks,Ad Impressions,Ad Spend,Ad Text,Age,Date,Interests,Language,Location,Number,Placements
0,0,137,44.87,Join us because we care. Black matters.,Age: 18 - 65+ Language: English (UK) or Englis...,06/10/15 02:59:53 AM PDT,N/A,Language: English (UK) or English (US) Placeme...,N/A,P(1)0000054.pdf,Placements: News Feed on desktop computers or ...
1,35,452,184.81,NOT EVERY BOY WANTS TO BE A SOLDIER. A beautif...,Age: 18 - 65+ Placements: News Feed on desktop...,06/23/15 07:04:01 AM PDT,N/A,N/A,N/A,P(1)0000180.pdf,Placements: News Feed on desktop computers or ...
2,N/A,N/A,N/A,NOT EVERY BOY WANTS TO BE A SOLDIER. A beautif...,Age: 18 - 65+ Placements: News Feed on desk...,N/A,N/A,N/A,N/A,P(1)0000182.pdf,N/A
3,0,31,33.59,?????? ??? ????? ? ??????????,Age: 18 - 65+ Language: English (UK) or Englis...,06/09/15 03:50:21 AM PDT,N/A,Language: English (UK) or English (US) Placeme...,N/A,P(1)0001844.pdf,"Placements: News Feed on desktop computers, Ne..."
4,4,326,45.94,California... knows how to party California......,Age: 18 - 65+ Placements: News Feed on desktop...,06/10/15 07:34:52 AM PDT,N/A,N/A,N/A,P(1)0002111.pdf,Placements: News Feed on desktop computers or ...
5,517,1,99.97,"Since 2010, over 350 of our lives have been ta...",Age: 18 - 65+ Placements: News Feed on desktop...,06/12/15 03:13:16 AM PDT,N/A,N/A,N/A,P(1)0002112.pdf,Placements: News Feed on desktop computers or ...
6,7,125,34.77,"'Just like Trayvon Martin, race mattered for A...",Age: 18 - 65+ Placements: News Feed on desktop...,06/11/15 06:51:30 AM PDT,N/A,N/A,N/A,P(1)0002113.pdf,Placements: News Feed on desktop computers or ...
7,17,168,31.54,Race war started by Texas teacher A Texas four...,Age: 18 - 65+ Placements: News Feed on desktop...,06/11/15 07:03:58 AM PDT,N/A,N/A,N/A,P(1)0002114.pdf,Placements: News Feed on desktop computers or ...
8,18,482,90.65,The image of 1938 shows several African Americ...,Age: 18 - 65+ Placements: News Feed on desktop...,06/15/15 07:21:33 AM PDT,N/A,N/A,N/A,P(1)0002115.pdf,Placements: News Feed on desktop computers or ...
9,24,524,88.45,American Racists On The Road The racists group...,Age: 18 - 65+ Placements: News Feed on desktop...,06/15/15 07:22:00 AM PDT,N/A,N/A,N/A,P(1)0002116.pdf,Placements: News Feed on desktop computers or ...


### Abspeichern als CSV

In [79]:
df.to_csv('readable.csv')